This is a rudimentary version of the signal processing algorithm

In [13]:
import mne
import yasa
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(style='white', font_scale=1.2)
from scipy.signal import welch
import math

We will need some way of reading in the data, and splitting it by 30 s epochs.

In [2]:
#Read-in raw data
#filename='sub-02_mne_raw.fif'
#raw= mne.io.read_raw_fif(filename, preload=True, verbose=0)
#Note: this will not work in real-time- I'm unsure how that will work
filename='ST7011J0-PSG.edf'
raw= mne.io.read_raw_edf(filename, preload=True, verbose=0)
times= raw.times
print(times)
sf = raw.info['sfreq']
tmax= times.size/sf
epoch_num=tmax//30
#for epoch in epoch_num:
    #raw.save('epoch'+epoch+'.fif', tmin=30*(epoch-1), tmax=30*epoch)
#Issue: this will save a separate file for each epoch, which is not efficient and we should avoid
print(epoch_num)
#raw.save('epoch1.fif', tmin=0, tmax=30, overwrite=True) #change to edf?
#Some variable may be needed to keep track of time
#At t=30, the raw data will be split off into its own array and a new array will be created for the next epoch
    #This will probably involve the use of an iterator variable
    #Maybe use 2D array?
#Filter the signal!!

[0.000000e+00 1.000000e-02 2.000000e-02 ... 3.589997e+04 3.589998e+04
 3.589999e+04]
1196.0


Combine the next 2 code blocks into one function, returns sleep stage for one epoch

In [44]:
def get_stage(data, sf):
    #data should be from mne file, in microvolts
    bp=yasa.bandpower(data, sf=sf) #channel names haven't been set- should be fine since we only have one channel
    #print(bp)
    alpha=bp.at['CHAN000', 'Alpha']
    if math.isnan(alpha):
        alpha=0
    print("Alpha:")
    print(alpha)
    beta=bp.at['CHAN000', 'Beta']
    if math.isnan(beta):
        beta=0
    print("Beta:")
    print(beta)
    theta=bp.at['CHAN000', 'Theta']
    if math.isnan(theta):
        theta=0
    print("Theta:")
    print(theta)
    #Incorrect data with zero signal
    #Slow wave detection
    slow=yasa.sw_detect(data, sf=sf, verbose='error')
    #Check the data types- can get errors if it is not microvolts
    #print(slow.summary(grp_chan=True,aggfunc='mean'))
    if slow==None:
        slow_num=0
        #print(slow_num)
    else:
        slow_num=(slow.summary(grp_chan=True,aggfunc='mean')).at['CHAN000', 'Count'] #May need to add paramters for this function
        #print(slow_num)
    print("Slow Wave:")
    print(slow_num)
    spindles=yasa.spindles_detect(data, sf=sf, verbose='error')
    #Suppress "no spindles detected" warning
    spin_threshold=1
    slow_threshold=1
    if spindles== None:
        spindles_num=0
        #print(spindles_num)
    else:
        spindles_shape=(spindles.summary()).size
        spindles_num=spindles_shape #In theory, spindles_shape will be the dimensions, and this will grab the number of rows
        print(spindles.summary())
        #Needs debugging!
    print("Spindles:")
    print(spindles_num)
    if (alpha+beta>0.5):
        if theta>0.25:
            stage='R'
        else:
            stage='W'
    else:
        if slow_num>slow_threshold: #Will need to set a threshold for # of slow waves in N3
            stage='N3'
        elif spindles_num>spin_threshold:
            stage='N2'
        else:
            stage='N1'
    return stage

In [23]:
#Make a loop to iterate through an entire file
epoch_num=2
stages= [0]*epoch_num
for i in range(epoch_num):
    print(i)
    #epoch_raw=raw.get_data(start=) #Need to make this a copy-by-value, or find another way to get subset
    #epoch_raw.crop(tmin=30*i, tmax=30*(i+1))
    #epoch_raw.pick_channels(['EEG Fpz-Cz'])
    #data=epoch1._data*1e6 #Changing units from V to microvolts
    raw.pick_channels(['EEG Fpz-Cz'])
    sf=raw.info['sfreq']
    data= raw.get_data(start=int(30*sf*i), stop=int(30*sf*(i+1)))*1e6
    #print(data.shape)
    #print(sf)
    stages[i]=get_stage(data,sf)
    print(stages[i])
print(stages)

0
Alpha:
0
Beta:
0
Theta:
0
Slow Wave:
1
Spindles:
0
N1
1
Alpha:
0.013073428649713456
Beta:
0.03253906794060435
Theta:
0.036557041034555676
Slow Wave:
0
Spindles:
0
N1
['N1', 'N1']


In [47]:
i=52
print(i)
#epoch_raw=raw.get_data(start=) #Need to make this a copy-by-value, or find another way to get subset
#epoch_raw.crop(tmin=30*i, tmax=30*(i+1))
#epoch_raw.pick_channels(['EEG Fpz-Cz'])
#data=epoch1._data*1e6 #Changing units from V to microvolts
raw.pick_channels(['EEG Fpz-Cz'])
sf=raw.info['sfreq']
data= raw.get_data(start=int(30*sf*i), stop=int(30*sf*(i+1)))*1e6
#print(data.shape)
#print(sf)
print(get_stage(data,sf))

52
Alpha:
0.06543872446516974
Beta:
0.18478517423802482
Theta:
0.1173518650783665
Slow Wave:
0
   Start   Peak    End  Duration  Amplitude       RMS  AbsPower  RelPower  \
0  14.11  14.28  14.98      0.87   29.65235  7.244203  1.445299  0.219085   

   Frequency  Oscillations  Symmetry  Channel  IdxChannel  
0  13.827038          11.0  0.193182  CHAN000           0  
Spindles:
13
N2


In [15]:
#Calculate bandpower or FFT power
#epoch1_raw=mne.io.read_raw_fif('epoch1.fif', preload=True, verbose=0)
#epoch1=epoch1_raw.pick_types(eeg=True)#This doesn't seem to be removing the non-EEG channels
epoch1=epoch1_raw.pick_channels(['EEG Fpz-Cz'])
data=epoch1._data*1e6 #Changing units from V to microvolts
bp=yasa.bandpower(data, sf=sf) #channel names haven't been set- should be fine since we only have one channel
#print(bp)
alpha=bp.at['CHAN000', 'Alpha']
#print(alpha)
beta=bp.at['CHAN000', 'Beta']
#print(beta)
theta=bp.at['CHAN000', 'Theta']
#Check if it works when 0 waves are detected
#Slow wave detection
slow=yasa.sw_detect(data, sf=sf)
#Check the data types- can get errors if it is not microvolts
#print(slow.summary(grp_chan=True,aggfunc='mean'))
if slow==None:
    slow_num=0
    print(slow_num)
else:
    slow_num=(slow.summary(grp_chan=True,aggfunc='mean')).at['CHAN000', 'Count'] #May need to add paramters for this function
    #print(slow_num)
spindles=yasa.spindles_detect(data, sf=sf)
#print(spindles)
if spindles== None:
    spindles_num=0
    print(spindles_num)
else:
    spindles_shape=(spindles.summary()).size()
    spindles_num=spindles_shape(1) #In theory, spindles_shape will be the dimensions, and this will grab the number of rows
    print(spindles_num)
    #This side of the loop still needs to be tested

12-May-21 16:38:55 | WARNING | No spindle were found in channel CHAN000.
12-May-21 16:38:55 | WARNING | No spindles were found in data. Returning None.


0


In [5]:
slow_threshold=1 #Placeholder value
spin_threshold=1
#stages=np.zeros(5, dtype='S5') #Check code properly later- this will need to set "stages" to have the same number of elements as epohcs
stages=[] #Try to pre-allocate space if possible
# One possible solution- create an array of maximum possible size, delete all empty elements once staging is done

i=0
#Wakefulness detection

if (alpha+beta>0.5):
    if theta>0.25:
        stages[i]='R'
    else:
        stages[i]='W'
else:
    if slow_num>slow_threshold: #Will need to set a threshold for # of slow waves in N3
        stages[i]='N3'
    elif spindles_num>spin_threshold:
        stages[i]='N2'
    else:
        #stages[i]='N1'
        stages.append('N1')

print(stages)

#Issues: detection doesn't seem to be correct, formatting on array- this will probably work better with integers

['N1']
